# Задача 2 <br>
Выполнила: Борисова Мария, 120М группа

Задание: необходимо написать скрипт, который будет вычислять и строить литологическую колонку с фиксированным, заданным шагом по глубине. 

В таблице три столбца ZK;ZP;LITO = кровля, подошва и тип породы.

In [1]:
import numpy as np
import pandas as pd

Загрузка данных

In [2]:
data_0 = pd.read_csv('well35.csv', encoding='cp1251', sep = ';')
data_0 = data_0.sort_values('ZK', axis=0)
data_0 = data_0.drop(data_0.index[-1],axis = 0)
data_0

,ZK,ZP,LITO
5,2635.00,2635.59,Породы глинисто-карбонатно-кремнистые
6,2635.59,2637.19,Породы глинисто-карбонатно-кремнистые
7,2637.19,2637.44,Породы глинисто-карбонатно-кремнистые
8,2637.44,2639.02,Породы глинисто-карбонатно-кремнистые
9,2639.02,2641.40,Породы глинисто-кремнистые
10,2641.40,2642.17,Породы глинисто-кремнистые
11,2642.17,2643.04,Породы глинисто-кремнистые
12,2643.04,2643.91,Породы глинисто-кремнистые
13,2643.91,2644.15,Радиоляриты
14,2644.15,2644.57,Породы глинисто-кремнистые


Подготовка литологии

In [7]:
dict_lito = dict()
for name in data_0['LITO']:
    dict_lito[name] = dict_lito.get(name, 0)
dict_lito

{'Породы глинисто-карбонатно-кремнистые': 0,
 'Породы глинисто-кремнистые': 0,
 'Радиоляриты': 0,
 'Породы керогеново-глинисто-кремнистые': 0,
 'Породы керогеново-карбонатно-глинисто-кремнистые': 0,
 'Породы керогеново-кремнисто-глинистые': 0,
 'Породы кремнисто-глинистые': 0,
 'Известняки': 0}

In [8]:
lito_interlayer = {'Породы глинисто-карбонатно-кремнистые': ' с глинисто-карбонатно-кремнистыми прослоями',
 'Породы глинисто-кремнистые': ' с глинисто-кремнистыми прослоями',
 'Радиоляриты': ' с радиоляритами',
 'Породы керогеново-глинисто-кремнистые': ' с керогеново-глинисто-кремнистыми прослоями',
 'Породы керогеново-карбонатно-глинисто-кремнистые': ' с керогеново-карбонатно-глинисто-кремнистыми прослоями',
 'Породы керогеново-кремнисто-глинистые': ' с керогеново-кремнисто-глинистыми прослоями',
 'Породы кремнисто-глинистые': ' с кремнисто-глинистыми прослоями',
 'Известняки': ' с прослоями известняков',
 np.nan: ''}

Функция масштабирования

In [9]:
def data_change(data, list_zk):
    for i in list_zk:
        data = data.drop(data[data.ZK == i].index,axis = 0)   
            
    data = data.sort_values('ZK', axis=0)
    return data

In [10]:
def lito_update(zk, lenght, data):

    if lenght == data.ZP[data.ZK == zk].values[0]:
        return data.LITO[data.ZK == zk].values[0], data
    
    if lenght < data.ZP[data.ZK == zk].values[0]:
        #деление слоев
        new_row = {'ZK': lenght, 'ZP': data.ZP[data.ZK == zk].values[0], 'LITO': data.LITO[data.ZK == zk].values[0]}
        data = data.append(new_row, ignore_index=True)
        data = data.sort_values('ZK', axis=0)
        data.loc[data.ZK == zk, 'ZP'] = lenght
        
        return data.LITO[data.ZK == zk].values[0], data
    
    else:
        #объединение слоев с удалением старого
        dict_lito = dict()
        list_zk = []
        zk1 = zk
        while lenght > data.ZP[data.ZK == zk].values[0]:
            
            dict_lito[data.LITO[data.ZK == zk].values[0]] = dict_lito.get(name, 0) + 1
            
            zk = data.ZP[data.ZK == zk].values[0]
            list_zk += [zk]
            if data.ZP[data.ZK == zk].values[0] == data.ZP[data.ZK.index[-1]]:
                break
                

        dict_lito[data.LITO[data.ZK == zk].values[0]] = dict_lito.get(name, 0) + 1
        
        sorted_tuple = [i for i,j in sorted(dict_lito.items(), key=lambda x: -x[1])]
        data.loc[data.ZK == zk1, 'LITO'] = sorted_tuple[0]
        if len(sorted_tuple) > 1:
            for i in range(1, len(sorted_tuple)):
                data.loc[data.ZK == zk1, 'LITO'] += lito_interlayer[sorted_tuple[i]]
         
        if lenght > data.ZP[data.ZK.index[-1]]:
            data.loc[data.ZK == zk1, 'ZP'] = data.ZP[data.ZK.index[-1]]
            data = data_change(data, list_zk)
            return 1, data
        else:
            data.loc[data.ZK == zk1, 'ZP'] = lenght
            data.loc[data.ZK == zk, 'ZK'] = lenght
        
        data = data_change(data, list_zk)
        
        return lito_update(lenght, lenght + scale, data)

In [11]:
scale = float(input('Введите масштаб в метрах: '))

data = data_0.copy()
ZK_new = data.ZK[data.ZK.index[0]]
thickness = round(ZK_new + scale, 2)

while thickness + scale < data.ZP[data.ZK.index[-1]]:
    numb, data = lito_update(ZK_new, thickness, data)
    if numb == 1:
        break
    ZK_new = thickness
    thickness = round(thickness + scale, 2)

pd.set_option('display.max_colwidth', None)
data

,ZK,ZP,LITO
5,2635.0,2640.00,Породы глинисто-карбонатно-кремнистые с глинисто-кремнистыми прослоями
9,2640.0,2645.00,Породы глинисто-кремнистые с радиоляритами
16,2645.0,2650.00,Породы глинисто-кремнистые с керогеново-глинисто-кремнистыми прослоями с радиоляритами с керогеново-карбонатно-глинисто-кремнистыми прослоями
27,2650.0,2655.00,Породы глинисто-кремнистые с керогеново-карбонатно-глинисто-кремнистыми прослоями с керогеново-кремнисто-глинистыми прослоями с кремнисто-глинистыми прослоями
2,2655.0,2657.13,Породы глинисто-кремнистые с прослоями известняков с кремнисто-глинистыми прослоями
